# Exponential Smoothing (Optimized Tuning) Function

# About 

This function was created to forecast using Holt-Winters Exponential Smoothing. It uses Level, Trend and Seasonality to predict future sales. Exponential Smoothing is a tried and true time-series model, very well suited for supply chain forecasting.

I built this to take in your series, seasonal periodicity, specifying multiplicative or additive trend, and specifying the number of splits via Time Series Cross Validation.

** Please note that n_splits must be only 2 if you do not have more than 2 seasonal cycles.** 

The optimization function searches over a predefined coarse grid to find the optimal alpha, beta, gamma coefficients on the train series, 

The model returned is fully fit using the optimal params (minimizing the Mean Absolute Error).

In the future, I would like to add more robust cross validation to this workflow. 

Thanks for reading! 

Jack

In [ ]:
#forecast_holt_winters optimized ETS function (forecasts at weekly frequency for 1 year ahead or n_periods)
 
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import optuna
import warnings
 
def forecast_holts_winters_opt(ts_data,
 seasonal_periods=52,
 trend = 'add',
 seasonal = 'add',
 damped_trend=False,
 show_plot=False,
 n_splits = 2, # by default, this doesn't do a true cross validation, (I don't have enough seasonal periods to do so in alot of data)
 n_iter=500):
   
    warnings.filterwarnings('ignore')
    #specify minimum training size of 2 seasonal cycles obs
    #min_train_size = seasonal_periods * 2
 
    tscv = TimeSeriesSplit(n_splits=n_splits)
 
    for train_index, test_index in tscv.split(ts_data):
 
      train_data, test_data = ts_data.iloc[train_index], ts_data.iloc[test_index]
 
      #define optima alpha (level), beta (trend), gamma (seasonal)
    def objective(trial):
        # Define the parameter space to be searched, using coarse grid to minimize overfitting risk. It is important that hyper coefs are capped at .5,.5,.3 respectively to prevent overfit model....
        alpha = trial.suggest_float('alpha', 0, .5, step=0.1)
        beta = trial.suggest_float('beta', 0, .5, step=0.1)
        gamma = trial.suggest_float('gamma', 0, .3, step=0.1)
 
            # Fit Holt-Winters Exponential Smoothing model
        model = ExponentialSmoothing(train_data,
            trend=trend, seasonal=seasonal,
            seasonal_periods=seasonal_periods,
            initialization_method="estimated",
            damped_trend=damped_trend)
           
        fit_model = model.fit(smoothing_level=alpha,
                                  smoothing_trend=beta,
                                  smoothing_seasonal=gamma)
 
            # Make predictions
        predictions = fit_model.predict(start=0,
                                            end=len(train_data)-1)
 
            # Calculate MAE and store it
        mae = mean_absolute_error(train_data, predictions)

        rmse = mean_squared_error(train_data,predictions,squared=False)
 
          # Return the average MAE across folds
        return mae
 
      # Create an Optuna study object and optimize the objective function
    optuna.logging.set_verbosity(optuna.logging.WARNING)
 
    study = optuna.create_study(direction='minimize')
 
    study.optimize(objective, n_trials=n_iter)
 
      # Get the best hyperparameters, and best Mean Abs Error
    best_params = study.best_params
 
    best_mae = study.best_value
 
      # Print the best hyperparameters
    print("Best Hyperparameters:", best_params)
 
      # Fit the model with the best hyperparameters on the entire dataset
    best_model = ExponentialSmoothing(ts_data,
                                        trend=trend,
                                          seasonal=seasonal,
                                            seasonal_periods=seasonal_periods,
                                            initialization_method="estimated")
 
    best_fit_model = best_model.fit(smoothing_level=best_params['alpha'],
                                      smoothing_slope=best_params['beta'],
                                      smoothing_seasonal=best_params['gamma'])
 
      # Make predictions with the best model
    best_predictions = best_fit_model.predict(start=len(train_data), end=len(train_data) + len(test_data) -1)
 
    model_bias = np.mean(best_predictions - test_data)
 
    model_rmse = mean_squared_error(best_predictions,test_data,squared=False)
 
    print(f'MODEL MAE: {round(best_mae, 2)}')
 
    print(f'MODEL MAE %: {round(best_mae / test_data.mean(), 2)}')
       
    print(f'MODEL Bias: {round(model_bias, 2)}')
 
    print(f'MODEL Bias %: {round(model_bias / test_data.mean(), 2)}')
 
    print(f'MODEL RMSE: {round(model_rmse)}')
      #if true then plots future forecast and predictions against test
    if show_plot == True:
            plt.style.use('seaborn-v0_8-bright')

            fig, ax = plt.subplots(figsize=(20,10))

            sns.lineplot(x=train_data.index,y=train_data, label='Train Data',ax=ax,linewidth=2)
 
            sns.lineplot(x=test_data.index,y=test_data, label='Test Data',ax=ax,linewidth=2)
 
            sns.lineplot(x=best_predictions.index,y=best_predictions, label='Predicted',ax=ax,linewidth = 2)

            plt.legend()
 
            plt.show()
  #returns best model summary object
    return best_fit_model